In [45]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/chloe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
def definition(mot):
    return wn.synsets(mot)[0].definition()

In [47]:
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk

def lesk(context_sentence, ambiguous_word, pos=None, synsets=None):
    context = set(context_sentence)
    if synsets is None:
        synsets = wn.synsets(ambiguous_word)

    if pos:
        synsets = [ss for ss in synsets if str(ss.pos()) == pos]

    if not synsets:
        return None

    _, sense = max(
    (len(context.intersection(ss.definition().split())), ss) for ss in synsets
    )
    return sense

In [48]:
ambiguous_word1 = "means"
ambiguous_word2 = "mean"
context_sentence1 = 'The train is the most ecological means of transport. By choosing the train you reduce your carbonfootprint.'
context_sentence2 = 'What does it mean to be a scientist ? To be a scientist mean to love knowledge and to discover newcorrelations between natural phenomena.'

print(lesk(context_sentence1, ambiguous_word1, pos=None, synsets=None))
print(lesk(context_sentence2, ambiguous_word2, pos=None, synsets=None))


Synset('means.n.01')
Synset('mean.v.07')


In [49]:
for ss in wn.synsets('means'):
    print(ss, ss.definition())

Synset('means.n.01') how a result is obtained or an end is achieved
Synset('means.n.02') an instrumentality for accomplishing some end
Synset('means.n.03') considerable capital (wealth or income)
Synset('mean.n.01') an average of n numbers computed by adding some function of the numbers and dividing by some function of n
Synset('mean.v.01') mean or intend to express or convey
Synset('entail.v.01') have as a logical consequence
Synset('mean.v.03') denote or connote
Synset('intend.v.01') have in mind as a purpose
Synset('mean.v.05') have a specified degree of importance
Synset('think_of.v.04') intend to refer to
Synset('mean.v.07') destine or designate for a certain purpose


In [50]:
# Les différences par rapport au mot "resistance" sont que la signification du mot dans les 2 contextes reste la même. Le mot mean est également un nom et un verbe. Les mots suivent une hiérarchie dans Wordnet.

In [78]:
def lire_fichier(chemin):
    with open(chemin, encoding = "utf-8") as f:
        chaine = f.read()
    return chaine
def decouper_en_mots(chaine):
    liste_mots = chaine.split()
    return liste_mots

In [79]:
import glob
liste_mots = ["war", "peace", "defense", "invasion", "force", "independence"]
for chemin in glob.glob('/Users/chloe/Documents/toto/textes1_2_3/*.txt'):
    chaine = lire_fichier(chemin)
    context = decouper_en_mots(chaine)
    for mots in liste_mots:
        print(chemin, mots,":",lesk(context, mots, pos=None, synsets=None),lesk(context, mots, pos=None, synsets=None).definition())

/Users/chloe/Documents/toto/textes1_2_3/3.txt war : Synset('war.n.02') a legal state created by a declaration of war and ended by official declaration during which the international rules of war apply
/Users/chloe/Documents/toto/textes1_2_3/3.txt peace : Synset('peace.n.04') the general security of public places
/Users/chloe/Documents/toto/textes1_2_3/3.txt defense : Synset('defense.n.11') a defendant's answer or plea denying the truth of the charges against him
/Users/chloe/Documents/toto/textes1_2_3/3.txt invasion : Synset('invasion.n.03') (pathology) the spread of pathogenic microorganisms or malignant cells to new sites in the body
/Users/chloe/Documents/toto/textes1_2_3/3.txt force : Synset('force_out.n.01') a putout of a base runner who is required to run; the putout is accomplished by holding the ball while touching the base to which the runner must advance before the runner reaches that base
/Users/chloe/Documents/toto/textes1_2_3/3.txt independence : Synset('independence.n.01'

In [ ]:
# Dans ce corpus de textes, ces mêmes mots changent de sens dans les différengts textes. Comme par exemple, le mot "defense" veut dire dans les 2 premiers textes le département
# de la défense des USA, et dans le 3ème texte, il veut dire la défense d'un pays.
# Pour le mot "independence", il évoque dans les 2 premiers textes une ville dans le Missouri aux USA, alors que dans le dernier texte, il renvoie à l'indépendence d'un pays.
# Pour le mot "invasion", il renvoie dans les 2 premiers textes à l'acte d'envahir un territoire. Dans le dernier texte, il revoie à la propagation de micro organismes pathogènes.
# Enfin pour le mot "peace", il revoie à l'absence de guerre dans les 2 premiers textes, alors que dans le dernier texte, il renvoie à la sécurité générale dans les lieux publics.
# Ces observations nous montrent en conclusion que lorsque l'on change le contexte d'un texte, les mots changent de sens.